In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
## Features to be used

# - Fatigue
#   Days since last match
#   Distance covered in the last x matches?

# - Home Team Form
#   Goals difference of home team in the last x matches    
#   Goals difference of home team in the last x home matches    
#   Average number of points gained by home team in the last x matches
#   Number of home matches won by home team in its last x home matches
#   Home Team Win streak  
#   Home Team Newly Promoted Team?

# - Away Team Form
#   Goals difference of away team in the last x matches  
#   Goals difference of away team in the last x away matches
#   Average number of points gained by away team in the last x matches
#   Number of away matches won by away team in its last x away matches
#   Away Team Win streak
#   Away Team Newly Promoted Team?

# - Home Team Performance Index
#   Home Defense Performance Index
#   Home Midfield Performance Index
#   Home Attack Performance Index

# - Away Team Performance Index
#   Away Defense Performance Index
#   Away Midfield Performance Index
#   Away Attack Performance Index

# - Betting Odds
#     B365H
#     B365D
#     B365A

# - Head to Head
#   Goals difference in previous x encounters

## Data Preparation

There are two main data sources for this project: www.football-data.co.uk/ and www.fbref.com/en/. 

Both of these two data sources provide match statistics data for Premier League matches. We will extract match statistics from both of these websites and integrate them into one DataFrame.

First we create an empty DataFrame that will store the integrated data from the various data sources.

In [5]:
df = pd.DataFrame()

### Data Source 1

www.football-data-co.uk

In [7]:
datasource1_df = pd.DataFrame()
seasons = ['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']

for season in seasons:
    # read csv file for match statistics
    temp_df = pd.read_csv(f'datasets/{season}/{season}-match-statistics.csv')

    # drop irrelvant columns
    temp_df.drop(temp_df.loc[:, 'BWH':].columns, axis=1, inplace = True)
    temp_df.drop(['Div', 'HTHG', 'HTAG', 'HTR', 'Referee', 'Time'], axis=1, inplace = True, errors='ignore')
    
    # concatenate df to datasource_1_df
    if datasource1_df.empty:
        datasource1_df = temp_df
    else:
        datasource1_df = pd.concat([datasource1_df, temp_df]).reset_index(drop=True)

In [14]:
# Make sure we have 5 x 380 = 1900 matches in the DataFrame
datasource1_df.shape

(1900, 21)

In [ ]:
datasource1_df['HomeTeam']

### Data Source 2
www.fbref.com/en

In [123]:
# read csv file for each team
arsenal_fixtures_17_18_df = pd.read_csv('datasets/2017-2018/arsenal-fixtures.csv')

# drop irrelvant columns
arsenal_fixtures_17_18_df.drop(['Time', 'Day', 'Attendance', 'Captain', 'Formation', 'Referee', 'Match Report', 'Notes'], axis=1, inplace = True)

# add new feature: 'DaysLastPlayed'
arsenal_fixtures_17_18_df['Date'] =  pd.to_datetime(arsenal_fixtures_17_18_df['Date'])
arsenal_fixtures_17_18_df['DaysLastPlayed'] = arsenal_fixtures_17_18_df['Date'] - arsenal_fixtures_17_18_df['Date'].shift(1)

# filter by Premier League home matches only
arsenal_fixtures_17_18_df = arsenal_fixtures_17_18_df[(arsenal_fixtures_17_18_df['Comp'] == 'Premier League') & (arsenal_fixtures_17_18_df['Venue'] == 'Home')]
arsenal_fixtures_17_18_df = arsenal_fixtures_17_18_df.reset_index(drop=True)

# rename features
arsenal_fixtures_17_18_df = arsenal_fixtures_17_18_df.rename(columns={'xG': 'HxG', 'xGA': 'AxG', 'Poss': 'HPoss', 'Opponent': 'AwayTeam'})

# add new feature: 'HomeTeam'
arsenal_fixtures_17_18_df['HomeTeam'] = 'Arsenal'

# drop home team column
arsenal_fixtures_17_18_df.drop(['Venue'], axis=1, inplace = True)

# add new feature: 'APoss'
arsenal_fixtures_17_18_df['APoss'] = 100 - arsenal_fixtures_17_18_df['HPoss']

arsenal_fixtures_17_18_df.head()

,Date,Comp,Round,Result,GF,GA,AwayTeam,HxG,AxG,HPoss,DaysLastPlayed,HomeTeam,APoss
0,2017-08-11,Premier League,Matchweek 1,W,4,3,Leicester City,2.5,1.5,68.0,5 days,Arsenal,32.0
1,2017-09-09,Premier League,Matchweek 4,W,3,0,Bournemouth,2.2,0.6,58.0,13 days,Arsenal,42.0
2,2017-09-25,Premier League,Matchweek 6,W,2,0,West Brom,2.2,0.9,69.0,5 days,Arsenal,31.0
3,2017-10-01,Premier League,Matchweek 7,W,2,0,Brighton,2.4,0.4,64.0,3 days,Arsenal,36.0
4,2017-10-28,Premier League,Matchweek 10,W,2,1,Swansea City,2.0,0.9,72.0,4 days,Arsenal,28.0


In [ ]:
# rename teams